Testando se o ambiente está funcionando

In [2]:
print('teste')

teste


Importando bibliotecas para explorar os diretórios do computador

In [3]:
from os import listdir
from os.path import isfile, join

Verificando todos os arquivos obtidos da fonta de dados

In [4]:
path = "E://dump"

dump_files = listdir(path)

FileNotFoundError: [Errno 2] No such file or directory: 'E://dump'

In [5]:
import pandas as pd
import numpy as np

Tendo certeza que é possível ler os dados do GHtorrent.

In [ ]:
#pd.read_csv(join(path, 'project_languages.csv'))

Um artigo que mostra a estrutura dos arquivos do GHtorrent https://gousios.org/pub/ghtorrent-dataset-toolsuite.pdf
misturando com esse aqui também https://github.com/ghtorrent/ghtorrent.org/blob/master/files/schema.pdf
Então irei adicionar o nome nas colunas, acabei retirando a coluna pullrequest_id pois ela não fazia sentido no contexto

# Filtrando os repositórios Open Source

Devido a grande quantidade de commits, não foi possível os utilizar como filtro de projetos open-source. Nesse caso, iremos utilizar a relação project_members para encontrar quais projetos são relevantes, ou seja, que possuem contribuintes além do dono.

In [6]:
df_project_members = pd.read_csv(join(path, 'project_members.csv'), names=["repo_id", "user_id", "created_at"])

In [ ]:
df_project_members

In [ ]:
df_count_members = df_project_members.groupby("repo_id", as_index=False)["user_id"].count()

In [ ]:
df_open_source_members = df_count_members[df_count_members["user_id"]>10]

In [ ]:
df_open_source_members

Devivo a datação dos dados iremos filtrar somente os membros a partir de 2019

In [ ]:
df_project_members = df_project_members.loc[
    (pd.to_datetime(df_project_members['created_at']) > pd.to_datetime("01-01-2019")) & 
    (df_project_members['repo_id'].isin(df_open_source_members['repo_id']))
    ]

In [ ]:
#transforma o campo de data em data

df_project_members['created_at'] = pd.to_datetime(df_project_members['created_at'])

In [ ]:
df_project_members

# Dataframe alvo

A partir do estudo Recommending GitHub Projects for Developer Onboarding podemos extrair alguns atributos a serem extraídos do dataset para que seja estudado o quão receptivo um repositório é, são eles:

- Crescimento do Projeto: Quantidade de commits do projeto no momento de iniciação
- Data de criação do projeto: Projetos mais novos tendem a atrair mais desenvolvedores
- Momento de primeira contribuição: quando o projeto começa a receber contribuidores
- Momento da última contribuição: se a equipe do projeto se manter estável significa que existe menos oportunidades de onboarding
- Momento do primeiro commit: primeiro commit do projeto, a partir de quando passa a aceitar contribuições
- Momento do último commit: se o projeto receber muitos commits, mas nao tiver tanto onboarding significa q esse projeto necessita de mais contribuintes

In [13]:
#objetivo final

df_onboardings = pd.DataFrame({}, columns=["repo_id", "user_id", "onboarding_date", "project_author_id", "project_created_at", "total_commits","first_onboarding", "last_onboarding", "first_commit", "last_commit"])

df_onboardings

,repo_id,user_id,onboarding_date,project_author_id,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit


In [ ]:
chunks_commits = pd.read_csv(join(path, 'commits.csv'), names=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"], chunksize=5000000)

Agora irei filtrar somente os commits dos projetos selecionados como open source após o ano de 2019

# Pegando os commits referentes aos repositórios classificados como Open Source

In [ ]:

#i=0
#for chunk in chunks_commits:
#        df_commit = chunk[chunk['project_id'].isin(df_project_members['repo_id'])]
#        df_commit.to_csv(f'E:/dump/open_source/open_source_commits_{i}.csv')
#        i += 1


In [ ]:
dump_open_source_files = listdir(join(path, 'open_source'))


# Descobrindo usuários iniciantes

Para determinar quais contribuições são referentes a iniciantes teremos que descobrir qual foi o primeiro commit de cada usuário e qual foi o repositório dele

testando um método de filtrar os primeiros commits em repositórios open source de cada usuário sem q o ID e o repo_id sejam abstraídos

In [ ]:
def get_min_date_row(group):
    min_row = group.loc[group['created_at'].idxmin()]
    return min_row



In [ ]:
# first_user_commit_by_chunks2 = []

# i=0
# for file in dump_open_source_files:
#     print("lendo arquivo: ", i)
#     df_commit = pd.read_csv(join(path,'open_source',file))
#     df_commit["created_at"] = pd.to_datetime(df_commit["created_at"])
#     first_user_commit_by_chunks2.append(df_commit.groupby('author_id').apply(get_min_date_row).reset_index(drop=True))
#     i += 1

In [ ]:
# df_first_user_commit2 = pd.concat(first_user_commit_by_chunks2)

# df_first_user_commit2

In [ ]:
# df_first_user_commit2.drop('Unnamed: 0', axis=1)

# df_first_user_commit2.to_csv('first_user_commit.csv')

In [ ]:
#df_first_user_commit2 = pd.read_csv('first_user_commit.csv')

In [ ]:
#df_first_user_contrib = df_first_user_commit2.groupby('author_id').apply(get_min_date_row).reset_index(drop=True)



In [ ]:
#df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0', axis=1)
#df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0.1', axis=1)

In [ ]:
#df_first_user_contrib.to_csv('first_user_contrib.csv')

In [ ]:
df_first_user_contrib = pd.read_csv("first_user_contrib.csv")

Agora iremos filtrar somente as primeiras contribuições de cada pessoa, já que estamos focando em pessoas iniciantes

In [ ]:
df_first_user_contrib = df_first_user_contrib.drop('Unnamed: 0', axis=1)
df_first_user_contrib = df_first_user_contrib[pd.to_datetime(df_first_user_contrib['created_at']) > pd.to_datetime('01-01-2019')]

In [ ]:
df_first_user_contrib

In [ ]:
df_project_members

In [ ]:

df_contrib_x_commits = df_project_members.merge(df_first_user_contrib, left_on=['repo_id', 'user_id'], right_on=['project_id', 'author_id'],  how='inner', suffixes=['_members', '_commits'])


In [ ]:
df_onboardings['repo_id'] = df_contrib_x_commits['repo_id']
df_onboardings['user_id'] = df_contrib_x_commits['author_id']
df_onboardings['onboarding_date'] = df_contrib_x_commits['created_at_members']

df_onboardings

In [ ]:
df_onboardings.to_csv("onboarding.csv")

# Pegando os detalhes do projeto

In [16]:
df_onboardings = pd.read_csv("onboarding.csv")

In [6]:
chunk_projects = pd.read_csv(join(path, "projects.csv"), index_col=False, chunksize=5000000  ,escapechar='\\', header=None, sep=",", quotechar='"',encoding="utf-8", names=["id", "url", "owner_id", "name", "descriptor", "language", "created_at", "forked_from", "deleted", "updated_at"], na_values="not available", on_bad_lines="warn") #, engine="python")

In [7]:
unique_projects = df_onboardings['repo_id'].unique()
projects = []

i=0
for chunk in chunk_projects:
    print("chunk ", i)
    projects.append(chunk[chunk['id'].isin(unique_projects)])
    i += 1
    
    

C:\Users\gusge\AppData\Local\Temp\ipykernel_25484\4076905464.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  for chunk in chunk_projects:


chunk  0


C:\Users\gusge\AppData\Local\Temp\ipykernel_25484\4076905464.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  for chunk in chunk_projects:


chunk  1


C:\Users\gusge\AppData\Local\Temp\ipykernel_25484\4076905464.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  for chunk in chunk_projects:


...

chunk  35


C:\Users\gusge\AppData\Local\Temp\ipykernel_25484\4076905464.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  for chunk in chunk_projects:


chunk  36


C:\Users\gusge\AppData\Local\Temp\ipykernel_25484\4076905464.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  for chunk in chunk_projects:


chunk  37


In [8]:
df_projects = pd.concat(projects)

In [19]:
df_onboarding_x_projects = df_onboardings.merge(df_projects[['id', 'owner_id', 'language', 'created_at']], left_on='repo_id', right_on='id', suffixes=['', '_project'])



In [24]:

df_onboardings['project_author_id'] = df_onboarding_x_projects['owner_id']
df_onboardings['project_language'] = df_onboarding_x_projects['language']
df_onboardings['project_created_at'] = df_onboarding_x_projects['created_at']
#df_onboardings = df_onboardings.drop('Unnamed: 0', axis=1)



In [26]:
df_onboardings.to_csv('onboarding.csv')

In [27]:
df_onboardings

,repo_id,user_id,onboarding_date,project_author_id,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit,project_language
0,19,9245591,2020-06-17 19:28:17,78,2009-01-05 14:00:43,NaN,NaN,NaN,NaN,NaN,Objective-C
1,19,33292409,2020-06-05 14:53:56,78,2009-01-05 14:00:43,NaN,NaN,NaN,NaN,NaN,Objective-C
2,4567,32547120,2020-01-04 17:08:41,21229,2011-12-23 04:52:33,NaN,NaN,NaN,NaN,NaN,JavaScript
3,6830,44244864,2019-04-19 01:58:41,8978,2012-02-21 19:46:55,NaN,NaN,NaN,NaN,NaN,Scala
4,8373,30552107,2020-06-05 20:09:57,19185,2011-05-10 07:41:19,NaN,NaN,NaN,NaN,NaN,Python
...,...,...,...,...,...,...,...,...,...,...,...
16626,206997704,66799493,2021-03-05 18:06:24,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#
16627,206997704,66965962,2021-03-05 18:08:01,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#
16628,206997704,66994499,2021-03-05 18:10:13,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#
16629,206997704,67981477,2021-03-05 18:08:24,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#


# Pegando a data da primeira e da última contribuição

In [67]:
df_onboardings = pd.read_csv("onboarding.csv", index_col=0)

repos = df_onboardings['repo_id'].unique()

In [8]:
df_project_members = pd.read_csv("project_members.csv", names=["repo_id", "user_id", "created_at"])

df_project_members = df_project_members[df_project_members['repo_id'].isin(repos)]

In [36]:
teste = df_onboardings[df_onboardings['repo_id'] == 4567]
teste_dono =  teste['project_author_id']

teste_dono.values[0]


np.int64(21229)

In [52]:
def get_min_date_row(group):
    repo = group['repo_id'].unique()[0]
    this_repo = df_onboardings[df_onboardings['repo_id'] == repo]
    owner = this_repo['project_author_id'].values[0]
    
    group.drop(group[ group['user_id'] == owner ].index, inplace=True)
    min_row = group.loc[group['created_at'].idxmin()]

    return min_row

In [46]:
def get_max_date_row(group):
    repo = group['repo_id'].unique()[0]
    this_repo = df_onboardings[df_onboardings['repo_id'] == repo]
    user_onboard = this_repo['user_id'].values[0]

    group.drop(group[ group['user_id'] == user_onboard ].index, inplace=True)
    min_row = group.loc[group['created_at'].idxmax()]

    return min_row

In [53]:

df_first_repo_contrib = df_project_members.groupby('repo_id').apply(get_min_date_row).reset_index(drop=True)

/tmp/ipykernel_6005/1145869671.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_first_repo_contrib = df_project_members.groupby('repo_id').apply(get_min_date_row).reset_index(drop=True)


In [54]:
df_first_repo_contrib

,repo_id,user_id,created_at
0,19,1715601,2009-01-05 13:00:43
1,4567,21449,2011-12-23 03:52:33
2,6830,689911,2013-12-24 12:02:05
3,8373,14445,2011-05-10 05:41:19
4,31779,1003,2011-10-03 19:46:00
...,...,...,...
2889,206575072,42505408,2021-03-03 10:47:37
2890,206679205,31181786,2021-03-04 02:45:49
2891,206799386,46623845,2021-03-04 16:49:39
2892,206955864,67981142,2021-03-05 12:55:47


In [48]:
df_last_repo_contrib = df_project_members.groupby('repo_id').apply(get_max_date_row).reset_index(drop=True)

/tmp/ipykernel_6005/1494079361.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_last_repo_contrib = df_project_members.groupby('repo_id').apply(get_max_date_row).reset_index(drop=True)


In [68]:
df_onboard_x_contribs = df_onboardings.merge(df_first_repo_contrib, left_on='repo_id', right_on='repo_id', suffixes=['', '_first_contrib'] )

df_onboard_x_contribs = df_onboard_x_contribs.merge(df_last_repo_contrib, left_on='repo_id', right_on='repo_id', suffixes=['', '_last_contrib'] )

In [69]:
df_onboard_x_contribs

,repo_id,user_id,onboarding_date,project_author_id,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit,project_language,user_id_first_contrib,created_at,user_id_last_contrib,created_at_last_contrib
0,19,9245591,2020-06-17 19:28:17,78,2009-01-05 14:00:43,NaN,NaN,NaN,NaN,NaN,Objective-C,1715601,2009-01-05 13:00:43,33292409,2020-06-05 14:53:56
1,19,33292409,2020-06-05 14:53:56,78,2009-01-05 14:00:43,NaN,NaN,NaN,NaN,NaN,Objective-C,1715601,2009-01-05 13:00:43,33292409,2020-06-05 14:53:56
2,4567,32547120,2020-01-04 17:08:41,21229,2011-12-23 04:52:33,NaN,NaN,NaN,NaN,NaN,JavaScript,21449,2011-12-23 03:52:33,40837416,2020-09-10 01:37:15
3,6830,44244864,2019-04-19 01:58:41,8978,2012-02-21 19:46:55,NaN,NaN,NaN,NaN,NaN,Scala,689911,2013-12-24 12:02:05,233554,2021-02-26 15:22:04
4,8373,30552107,2020-06-05 20:09:57,19185,2011-05-10 07:41:19,NaN,NaN,NaN,NaN,NaN,Python,14445,2011-05-10 05:41:19,53925354,2021-02-02 19:09:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16626,206997704,66799493,2021-03-05 18:06:24,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#,67993950,2021-03-05 18:06:15,67982813,2021-03-05 18:40:18
16627,206997704,66965962,2021-03-05 18:08:01,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#,67993950,2021-03-05 18:06:15,67982813,2021-03-05 18:40:18
16628,206997704,66994499,2021-03-05 18:10:13,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#,67993950,2021-03-05 18:06:15,67982813,2021-03-05 18:40:18
16629,206997704,67981477,2021-03-05 18:08:24,3749802,2021-03-05 16:49:43,NaN,NaN,NaN,NaN,NaN,C#,67993950,2021-03-05 18:06:15,67982813,2021-03-05 18:40:18


In [70]:
df_onboardings['first_onboarding'] = df_onboard_x_contribs['created_at'] 
df_onboardings['first_onboarding_user_id'] = df_onboard_x_contribs['user_id_first_contrib']

df_onboardings['last_onboarding'] = df_onboard_x_contribs['created_at_last_contrib']
df_onboardings['last_onboarding_user_id'] = df_onboard_x_contribs['user_id_last_contrib']

df_onboardings

,repo_id,user_id,onboarding_date,project_author_id,project_created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit,project_language,first_onboarding_user_id,last_onboarding_user_id
0,19,9245591,2020-06-17 19:28:17,78,2009-01-05 14:00:43,NaN,2009-01-05 13:00:43,2020-06-05 14:53:56,NaN,NaN,Objective-C,1715601,33292409
1,19,33292409,2020-06-05 14:53:56,78,2009-01-05 14:00:43,NaN,2009-01-05 13:00:43,2020-06-05 14:53:56,NaN,NaN,Objective-C,1715601,33292409
2,4567,32547120,2020-01-04 17:08:41,21229,2011-12-23 04:52:33,NaN,2011-12-23 03:52:33,2020-09-10 01:37:15,NaN,NaN,JavaScript,21449,40837416
3,6830,44244864,2019-04-19 01:58:41,8978,2012-02-21 19:46:55,NaN,2013-12-24 12:02:05,2021-02-26 15:22:04,NaN,NaN,Scala,689911,233554
4,8373,30552107,2020-06-05 20:09:57,19185,2011-05-10 07:41:19,NaN,2011-05-10 05:41:19,2021-02-02 19:09:02,NaN,NaN,Python,14445,53925354
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16626,206997704,66799493,2021-03-05 18:06:24,3749802,2021-03-05 16:49:43,NaN,2021-03-05 18:06:15,2021-03-05 18:40:18,NaN,NaN,C#,67993950,67982813
16627,206997704,66965962,2021-03-05 18:08:01,3749802,2021-03-05 16:49:43,NaN,2021-03-05 18:06:15,2021-03-05 18:40:18,NaN,NaN,C#,67993950,67982813
16628,206997704,66994499,2021-03-05 18:10:13,3749802,2021-03-05 16:49:43,NaN,2021-03-05 18:06:15,2021-03-05 18:40:18,NaN,NaN,C#,67993950,67982813
16629,206997704,67981477,2021-03-05 18:08:24,3749802,2021-03-05 16:49:43,NaN,2021-03-05 18:06:15,2021-03-05 18:40:18,NaN,NaN,C#,67993950,67982813


In [71]:
df_onboardings.to_csv('onboarding.csv')

# Calculando a quantidade de commits no projeto no momento de onboarding

In [4]:
dump_open_source_files = listdir('open_source')


In [32]:
df_onboardings = pd.read_csv("onboarding.csv", index_col=0)


repos = df_onboardings['repo_id'].unique()

In [12]:
repos_commits = {}

for repo in repos:
        repos_commits[repo] = []

repos_commits['failed'] = []



In [15]:
for i, file in enumerate(dump_open_source_files):
        print("file: ", i)
        chunk = pd.read_csv(join('open_source', file))
        chunk_repos = chunk['project_id'].unique()
        for repo in chunk_repos:
                try:
                        repos_commits[repo].append(chunk[chunk['project_id'] == repo])
                except:
                        repos_commits['failed'].append(repo)

file:  0
file:  1
file:  2
file:  3
file:  4
file:  5
file:  6
file:  7
file:  8
file:  9
file:  10
file:  11
file:  12
file:  13
file:  14
file:  15
file:  16
file:  17
file:  18
file:  19
file:  20
file:  21
file:  22
file:  23
file:  24
file:  25
file:  26
file:  27
file:  28
file:  29
file:  30
file:  31
file:  32
file:  33
file:  34
file:  35
file:  36
file:  37
file:  38
file:  39
file:  40
file:  41
file:  42
file:  43
file:  44
file:  45
file:  46
file:  47
file:  48
file:  49
file:  50
file:  51
file:  52
file:  53
file:  54
file:  55
file:  56
file:  57
file:  58
file:  59
file:  60
file:  61
file:  62
file:  63
file:  64
file:  65
file:  66
file:  67
file:  68
file:  69
file:  70
file:  71
file:  72
file:  73
file:  74
file:  75
file:  76
file:  77
file:  78
file:  79
file:  80
file:  81
file:  82
file:  83
file:  84
file:  85
file:  86
file:  87
file:  88
file:  89
file:  90
file:  91
file:  92
file:  93
file:  94
file:  95
file:  96
file:  97
file:  98
file:  99
file:  100

In [ ]:
for repo in repos_commits:
    df_repo_commits = pd.concat(repos_commits[repo])
    if repo != 'failed':
        df_repo_commits.to_csv(join('repos_commits', 'repos_commits_{0}.csv'.format(repo)))

In [46]:
total_commits = []
first_commits = []
last_commits = []

def get_total_commits(row):
    repo_id = row['repo_id']
    date = row['onboarding_date']

    project_commits = pd.read_csv(join('repos_commits', 'repos_commits_{0}.csv'.format(repo_id)))
    project_commits['created_at'] = pd.to_datetime(project_commits['created_at'])

    project_commits = project_commits.loc[project_commits['created_at'] < pd.to_datetime(date)]

    total_commits.append(project_commits.shape[0])
    first_commits.append(project_commits['created_at'].min())
    last_commits.append(project_commits['created_at'].max())

In [47]:

df_onboardings.apply(get_total_commits, axis=1)   

df_onboardings['total_commits'] = total_commits
df_onboardings['first_commit'] = first_commits
df_onboardings['last_commit'] = last_commits

/tmp/ipykernel_7002/503111727.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  project_commits['created_at'] = pd.to_datetime(project_commits['created_at'])
/tmp/ipykernel_7002/503111727.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  project_commits['created_at'] = pd.to_datetime(project_commits['created_at'])
/tmp/ipykernel_7002/503111727.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  project_commits['created_at'] = pd.to_datetime(project_commits['created_at'])
/tmp/ipykernel_7002/503111727.py:10: UserWarning: Could not infer format, so each element will be

In [50]:
df_onboardings.to_csv('onboarding.csv')